In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv

import time 
import os

# !ls
os.chdir('/content/drive/')

start_time = time.time()

TRAIN_DATA_PATH = 'nn/smart_city/train_data/scene1_jiading_lib_training/scene1_jiading_lib_training_coordinates.csv'
FEATURE_PATH = 'nn/smart_city/feature_scene1.csv'
CHECKPOINT_PATH = 'nn/smart_city/models/scene1'
MODEL_PATH = os.path.join(CHECKPOINT_PATH, 'scene1_model.meta')
RESULT_PATH = 'nn/smart_city/result_scene1.csv'

if __name__ == '__main__':
  features = pd.read_csv(FEATURE_PATH)
  features.columns = ['dir_name', 'f_x', 'f_y']
  
  # 求标记的平均值
  average = np.array([0, 0, 0])
  train_data = pd.read_csv(TRAIN_DATA_PATH)
  
  for row in train_data.values:
    average = average + row[1:]
    
  average /= train_data.shape[0]
  
  with tf.Session() as sess:
    # 加载模型参数
    saver = tf.train.import_meta_graph(MODEL_PATH)
    saver.restore(sess, tf.train.latest_checkpoint(CHECKPOINT_PATH))
    
    W1 = sess.run('W1:0')
    b1 = sess.run('b1:0')
    W2 = sess.run('W2:0')
    b2 = sess.run('b2:0')
    
    # 输出结果
    with open(RESULT_PATH, 'w') as f:
      # 将DataFrame 按 dir_name 划分成多个小的小 DataFrame
      dir_list = features['dir_name'].unique().tolist()
      for dir in dir_list:
        feature = features.loc[features.dir_name == dir]
        data = feature.drop(['dir_name'], axis=1, inplace=False)
        
        X = tf.Variable(tf.cast(data, dtype=tf.float32))
        sess.run(tf.global_variables_initializer()) # 初始化定义的tensorflow变量

        # 运行模型
        temp = tf.matmul(X, W1) + b1
        _y = tf.matmul(temp, W2) + b2
        
        _y = _y.eval()
        coordinate = _y.sum(axis=0) / _y.shape[0] + average
        row = list(coordinate)
        row.insert(0, dir)
        
        writer = csv.writer(f)
        writer.writerow(row)

    print('time = %ss'%(time.time() - start_time))
    
# 参考
# 1、Python的numpy库中将矩阵转换为列表等函数：https://blog.csdn.net/flying_sfeng/article/details/66478616
# 2、Splitting dataframe into multiple dataframes：https://stackoverflow.com/questions/19790790/splitting-dataframe-into-multiple-dataframes

time = 126.8646457195282s
